In [ ]:
pip install pya3

In [26]:
from pya3 import *
import datetime
from datetime import time

In [51]:
userid='579619'
apikey='dHFZCVGMMAbCn2xDPWVqi8ohtHeDNcL8pERFqto0FfiYdzpdea9ZjNMJUKt7bYgRCd7PrvqgF6QdJlG8yavGFmRPZNHmEoPnIWmLNtUQRERAEpc0dYYAfJ87AWOdCkRW'

from datetime import datetime

alice = Aliceblue(user_id=userid,api_key=apikey)


print(alice.get_session_id())

#print(alice.get_balance())

#print(alice.get_profile())

LTP = 0
socket_opened = False
subscribe_flag = False
subscribe_list = []
unsubscribe_list = []
ltp_price=0

def socket_open():  # Socket open callback function
    print("Connected")
    global socket_opened
    socket_opened = True
    if subscribe_flag:  # This is used to resubscribe the script when reconnect the socket.
        alice.subscribe(subscribe_list)

def socket_close():  # On Socket close this callback function will trigger
    global socket_opened, LTP
    socket_opened = False
    LTP = 0
    print("Closed")

def socket_error(message):  # Socket Error Message will receive in this callback function
    global LTP
    LTP = 0
    print("Error :", message)

def feed_data(message):  # Socket feed data will receive in this callback function
    global LTP, subscribe_flag
    feed_message = json.loads(message)
    if feed_message["t"] == "ck":
        print("Connection Acknowledgement status :%s (Websocket Connected)" % feed_message["s"])
        subscribe_flag = True
        print("subscribe_flag :", subscribe_flag)
        print("-------------------------------------------------------------------------------")
        pass
    elif feed_message["t"] == "tk":
        print("Token Acknowledgement status :%s " % feed_message)
        print(LTP)
        print("here is the ltp price")
        print(feed_message['lp'])
        ltp_price=feed_message['lp']
        ltp_price=float( ltp_price)   
        ltp_price=int( ltp_price) 
        print(type(ltp_price))
    
        get_strike_price(ltp_price)
        
        print("-------------------------------------------------------------------------------")
        pass
    else:
        print("Feed :", feed_message)
        LTP = feed_message['lp'] if 'lp' in feed_message else LTP  # If LTP in the response it will store in LTP variable
        print(LTP)
        #print(type(LTP))
        print("-------------------------------------------------------------------------------")
        
def get_strike_price(ltp):
    ce_strike=round(ltp/500)*500
    pe_strike=round(ltp/500)*500
    print("ce",ce_strike)
    print("pe",pe_strike)
    #getting bn call and put objects
    get_call_but_objects(ce_strike,pe_strike)
    
   
def get_call_but_objects(ce_strike,pe_strike):
    #exch="NFO",symbol='BANKNIFTY', expiry_date="2022-09-25", is_fut=True,strike=None, is_CE=False
    bn_call=alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date="2023-03-29", is_fut=False,strike=ce_strike, is_CE=True)
    bn_put=alice.get_instrument_for_fno(exch="NFO",symbol='BANKNIFTY', expiry_date="2023-03-29", is_fut=False,strike=pe_strike, is_CE=False)
    #print(bn_call)
    #placing ce and pe orders
    print("Placing orders")
    place_ce_order(bn_call)
    place_ce_order(bn_put)
    
def place_ce_order(bn_call):
    sell_order = alice.place_order(transaction_type = TransactionType.Sell,
                             instrument = bn_call,
                             quantity = 25,
                             order_type = OrderType.Market,
                             product_type = ProductType.Intraday,
                             price = 0.0,
                             trigger_price = None,
                             stop_loss = None,
                             square_off = None,
                             trailing_sl = None,
                             is_amo = False)
    print("-------------------------------------------------------------------------------")
    print("CE sell order details : ",sell_order)
    
    
def place_pe_order(bn_put):
    sell_order = alice.place_order(transaction_type = TransactionType.Sell,
                             instrument = bn_put,
                             quantity = 25,
                             order_type = OrderType.Market,
                             product_type = ProductType.Intraday,
                             price = 0.0,
                             trigger_price = None,
                             stop_loss = None,
                             square_off = None,
                             trailing_sl = None,
                             is_amo = False)
    print("-------------------------------------------------------------------------------")
    print("CE sell order details :",sell_order)
    


target_time=time(12,11,0)
while datetime.now().time()<target_time:
    print("-------------------------------------------------------------------------------")
    print("the orders will be placed at 1PM")
    #print(datetime.datetime.now().time)
    sleep(1)

print("-------------------------------------------------------------------------------")
print("Logged in")
# Socket Connection Request
alice.start_websocket(socket_open_callback=socket_open, socket_close_callback=socket_close,
                      socket_error_callback=socket_error, subscription_callback=feed_data, run_in_background=True,market_depth=False)

while not socket_opened:
    pass

subscribe_list = [alice.get_instrument_by_token('INDICES', 26009)]
alice.subscribe(subscribe_list)
print(datetime.now())
sleep(2)
print(datetime.now())
# unsubscribe_list = [alice.get_instrument_by_symbol("NSE", "RELIANCE")]
# alice.unsubscribe(unsubscribe_list)
# sleep(8)

# Stop the websocket
alice.stop_websocket()
#sleep(10)
#print(datetime.now())

# Connect the socket after socket close
#alice.start_websocket(socket_open_callback=socket_open, socket_close_callback=socket_close,
                      #socket_error_callback=socket_error, subscription_callback=feed_data, run_in_background=True)

{'stat': 'Ok', 'sessionID': 'TMjXQZ6LLcY9GL9PibDyQkotya95JAN6QLJqRhM51cTtPnzEAdkiyQfTiy0oh99I6hf5heIE7W19jbp37Ttn6779QuyzH6kDS35Rict7dyNNEzcFzGhnApcKPIpT4AiTG6kmOtnE9WtXbRRmgGFGgWsxrMW2jrB7bZMKvOHffD0u9xezg5hmKt7vxc9pgBWPopVDTieUEU39RzBABOBt4qhNPBZ32tV0X1hvMqapC7or74AgIqenrYXODdJyCQZT'}
-------------------------------------------------------------------------------
Logged in
STAGE 1: Invalidate the previous session : Ok
STAGE 2: Create the new session : Ok
Connecting to Socket ...
Connected
2023-03-18 12:19:32.235838
Connection Acknowledgement status :OK (Websocket Connected)
subscribe_flag : True
-------------------------------------------------------------------------------
Token Acknowledgement status :{'t': 'tk', 'pp': '2', 'ml': '1', 'e': 'NSE', 'tk': '26009', 'ls': '1', 'ti': '0.05', 'c': '39598.10', 'lp': '39784.15', 'pc': '0.47', 'ft': '1679122171', 'o': '39176.35', 'h': '40826.55', 'l': '38664.25'} 
0
here is the ltp price
39784.15
<class 'int'>
ce 40000
pe 40000
Placing order

In [ ]:
#getting expiry date 
#getting ce and pe prices and placing orders thrrough them
#placing sl orders